---
---
# NEURAL NETWORKS AND DEEP LEARNING

### A.A. 2021/22 (6 CFU) - Dr. Alberto Testolin, Dr. Umberto Michieli

### Saverio Monaco
##### MAT: 2012264

# Homework 3 - Deep Reinforcement Learning
---
---

In [1]:
#################
#### IMPORTS ####
#################

# Arrays
import numpy as np
from collections import deque # fixed size FIFO list

# Deep Learning Stuff
import torch
from torch import nn
import gym

# Visualizing
import matplotlib.pyplot as plt
from gym.wrappers import Monitor
from IPython.display import HTML
from IPython import display as ipythondisplay
import base64

# Other
import random
from tqdm.notebook import tqdm
import io
import os
import glob

# Automatic tuning tool
import optuna

# Second set
import flappy_bird_gym
import time

In [2]:
# Set random seeds for reproducibility
torch.manual_seed(0)
np.random.seed(0)
random.seed(0)


In [3]:
# Check if the GPU is available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Training device: {device}")


Training device: cuda


In [4]:
device = 'cpu'

In [5]:
#################
#### CLASSES ####
#################

class ReplayMemory(object):
    '''
    To perform experience replay.
    We will draw uniformly at random from the pool of stored sample to learn.
    Thi avoids (temporal) correlation between consecutive learning instances.
    '''
    def __init__(self, capacity):
        ''' Initialize a deque with maximum capacity maxlen. '''
        self.memory = deque(maxlen=capacity) # Define a queue with maxlen "capacity"

    def push(self, state, action, next_state, reward):
        ''' Add a new sample to the deque, removes the oldest one if it is already full. '''
        # Add the tuple (state, action, next_state, reward) to the queue
        self.memory.append( (state, action, next_state, reward) )
        
    def sample(self, batch_size):
        ''' Randomly select "batch_size" samples '''
        batch_size = min(batch_size, len(self)) # Get all the samples if the requested batch_size is higher than the number of sample currently in the memory
        return random.sample(self.memory, batch_size)

    def __len__(self):
        ''' Return the number of samples currently stored in the memory '''
        return len(self.memory)

class DQN(nn.Module):
    ''' 
    Network for policy network and target network 
    state_space_dim:  (INPUT)  dimension of state space (e.g pixels in a image)
    action_space_dim: (OUTPUT) dimension of action space (e.g go left, go right)
    '''
    def __init__(self, DQN_state_space_dim, DQN_action_space_dim):
        super().__init__()
            
        self.sdim = DQN_state_space_dim
        self.adim = DQN_action_space_dim
            
        self.linear = nn.Sequential(
            nn.Linear(self.sdim, 128),
            nn.Tanh(),
            nn.Linear(128,128),
            nn.Tanh(),
            nn.Linear(128,self.adim)
                )

    def forward(self, x):
        return self.linear(x)
    
#
#          |---------> [Prediction Network (DQN)]--------
#          |                |                            \
# [INPUT]--|                | Parameter update            \___Loss
#          |               \/                            /
#          |---------> [Target Network (DQN)]------------
#
class FullQNets(nn.Module):
    ''' 
    Handles all the networks, environments, and others
    '''
    def __init__(self, envname):
        super().__init__()
        
        self.envname = envname
        
        tempenv = gym.make(self.envname) 
            
        self.state_space_dim = tempenv.observation_space.shape[0]
        
        if self.envname == 'BipedalWalkerHardcore-v3':
            self.action_space_dim = 4 # Box attribute in BipedalWalker are different
        else:
            self.action_space_dim = tempenv.action_space.n
        
        self.policy_net = DQN(self.state_space_dim, self.action_space_dim)
        self.target_net = DQN(self.state_space_dim, self.action_space_dim)
        self.target_net.load_state_dict(self.policy_net.state_dict())
    
    def choose_action_epsilon_greedy(self, state, epsilon):
        self.policy_net.eval()
        if epsilon > 1 or epsilon < 0:
            raise Exception('The epsilon value must be between 0 and 1')
                
        # Evaluate the network output from the current state
        with torch.no_grad():
            self.policy_net.eval()
            state = torch.tensor(state, dtype=torch.float32, device=device) # Convert the state to tensor
            net_out = self.policy_net(state)

        # Get the best action (argmax of the network output)
        best_action = int(net_out.argmax())
        # Get the number of possible actions
        action_space_dim = net_out.shape[-1]

        # Select a non optimal action with probability epsilon, otherwise choose the best action
        if random.random() < epsilon:
            # List of non-optimal actions
            non_optimal_actions = [a for a in range(action_space_dim) if a != best_action]
            # Select randomly
            action = random.choice(non_optimal_actions)
        else:
            # Select best action
            action = best_action
        
        return action, net_out.cpu().numpy()
    
    def choose_action_softmax(self, state, temperature):
        self.policy_net.to(device)
        if temperature < 0:
            raise Exception('The temperature value must be greater than or equal to 0 ')
        
        # If the temperature is 0, just select the best action using the eps-greedy policy with epsilon = 0
        if temperature == 0:
            return self.choose_action_epsilon_greedy(state, 0)
    
        # Evaluate the network output from the current state
        with torch.no_grad():
            self.policy_net.eval()
            state = torch.tensor(state, dtype=torch.float32, device=device)
            net_out = self.policy_net(state)

        # Apply softmax with temp
        temperature = max(temperature, 1e-8) # set a minimum to the temperature for numerical stability
        softmax_out = nn.functional.softmax(net_out / temperature, dim=0).cpu().numpy()
                
        # Sample the action using softmax output as mass pdf
        all_possible_actions = np.arange(0, softmax_out.shape[-1])
        action = np.random.choice(all_possible_actions, p=softmax_out) # this samples a random element from "all_possible_actions" with the probability distribution p (softmax_out in this case)
    
        return action, net_out.cpu().numpy()
    
    def update_step(self, replay_mem, gamma, optimizer, loss_fn, batch_size):        
        self.policy_net.to(device)
        self.target_net.to(device)
        # Sample the data from the replay memory
        batch = replay_mem.sample(batch_size)
        batch_size = len(batch)

        # Create tensors for each element of the batch
        states      = torch.tensor([s[0] for s in batch], dtype=torch.float32, device=device)
        actions     = torch.tensor([s[1] for s in batch], dtype=torch.int64, device=device)
        rewards     = torch.tensor([s[3] for s in batch], dtype=torch.float32, device=device)

        # Compute a mask of non-final states (all the elements where the next state is not None)
        non_final_next_states = torch.tensor([s[2] for s in batch if s[2] is not None], dtype=torch.float32, device=device) # the next state can be None if the game has ended
        non_final_mask = torch.tensor([s[2] is not None for s in batch], dtype=torch.bool, device=device)

        # Compute all the Q values (forward pass)
        self.policy_net.train()
        q_values = self.policy_net(states)
        # Select the proper Q value for the corresponding action taken Q(s_t, a)
        state_action_values = q_values.gather(1, actions.unsqueeze(1))
        
        # Compute the value function of the next states using the target network V(s_{t+1}) = max_a( Q_target(s_{t+1}, a)) )
        with torch.no_grad():
            self.target_net.eval()
            q_values_target = self.target_net(non_final_next_states)
        next_state_max_q_values = torch.zeros(batch_size, device=device)
        next_state_max_q_values[non_final_mask] = q_values_target.max(dim=1)[0]

        # Compute the expected Q values
        expected_state_action_values = rewards + (next_state_max_q_values * gamma)
        expected_state_action_values = expected_state_action_values.unsqueeze(1) # Set the required tensor shape

        # Compute the Huber loss
        loss = loss_fn(state_action_values, expected_state_action_values)

        # Optimize the model
        optimizer.zero_grad()
        loss.backward()
        # Apply gradient clipping (clip all the gradients greater than 2 for training stability)
        nn.utils.clip_grad_norm_(self.policy_net.parameters(), 2)
        optimizer.step()
        
        return optimizer
    
    def play_a_game(self, show = True, debug = False):
        # Initialize the Gym environment
        env = gym.make(self.envname) 
        
        # Reset the environment and get the initial state
        state = env.reset()
        # Reset the score. The final score will be the total amount of steps before the pole falls
        score = 0
        done = False
        # Go on until the pole falls off or the score reach 490
        while not done:
            # Choose the best action (temperature 0)
            action, q_values = self.choose_action_softmax(state, temperature=0)
            if debug:
                print('action', action)
                print('state', state)
            # Apply the action and get the next state, the reward and a flag "done" that is True if the game is ended
            next_state, reward, done, info = env.step(action)
            # Visually render the environment
            if show:
                env.render()
            # Update the final score (+1 for each step)
            score += reward 
            # Set the current state for the next iteration
            state = next_state
            # Check if the episode ended (the pole fell down)
        # Print the final score
        if show:
            print(f"SCORE: {score}") 
        env.close()
        
        return score
    
    def training_loop(self, max_epoch, loss_fn, optimizer_fn, initial_value=5, gamma = 0.97, 
                 replay_memory_capacity = 10000, lr = 1e-2, reg_opt = 1e-5, 
                 target_net_update_steps = 10, bad_state_penalty = 0, test_policy = False,
                 batch_size = 128, min_samples_for_training = 1000, render=False, optunaprint = False, maxplayers = 0,
                 optuna_epoch_limit = False):
        '''
        PARAMETERS
        gamma: gamma parameter for the long term reward
        replay_memory_capacity: Replay memory capacity
        lr: Optimizer learning rate
        loss_fn: Loss function
        optimizer_fn: Optimizer function
        target_net_update_steps: Number of episodes to wait before updating the target network
        batch_size: Number of samples to take from the replay memory for each update
        bad_state_penalty: Penalty to the reward when we are in a bad state (in this case when the pole falls down) 
        min_samples_for_training: Minimum samples in the replay memory to enable the training
        test_policy: if True, for every target_net_update_steps steps, it will play 10 games following its policy (T=0)
                     and if it obtains the maximum score (500) it will exit
                     if is int > 0 it will do the same playing test_policy number of games
        '''
        env = gym.make(self.envname)
        optimizer = optimizer_fn(self.policy_net.parameters(), lr=lr, weight_decay=reg_opt)
        has_learned = False # Boolean variable, if test_policy == True and has obtained the max score on the test games
                            # it will exit learning
        # We compute the exponential decay in such a way the shape of the exploration 
        # profile does not depend on the number of iterations
        exp_decay = np.exp(-np.log(initial_value) / max_epoch * 6) 
        exploration_profile = [initial_value * (exp_decay ** i) for i in range(max_epoch)]
        
        ### Initialize the replay memory
        replay_mem = ReplayMemory(replay_memory_capacity)   
        
        env.seed(0)
        policy_num = 0
        progress = tqdm(exploration_profile)
        #print('n iterations:', len(exploration_profile) )
        for episode_num, tau in enumerate(progress):
            # Reset the environment and get the initial state
            state = env.reset()
            
            # Reset the score. The final score will be the total amount of steps before the pole falls
            score = 0
            
            done = False
            
            # Go on until the pole falls off
            while not done:
                # Choose the action following the policy
                action, q_values = self.choose_action_softmax(state, temperature=tau)
                
                # Apply the action and get the next state, the reward and a flag "done" 
                # that is True if the game is ended
                next_state, reward, done, info = env.step(action)
                
                # We apply a (linear) penalty when the cart is far from center
                pos_weight = 1
                reward = reward - pos_weight * np.abs(state[0])
                
                # Update the final score (+1 for each step)
                score += 1
                
                # Update the replay memory
                replay_mem.push(state, action, next_state, reward)

                # Update the network
                if len(replay_mem) > min_samples_for_training: # we enable the training only if we have enough samples in the replay memory, otherwise the training will use the same samples too often
                    optimizer = self.update_step(replay_mem, gamma, optimizer, loss_fn, batch_size)
                
                if render:
                    # Visually render the environment (disable to speed up the training)
                    env.render()

                # Set the current state for the next iteration
                state = next_state

            # Update the target network every target_net_update_steps episodes
            if episode_num % target_net_update_steps == 0:
                self.target_net.load_state_dict(self.policy_net.state_dict()) # This will copy the weights of the policy network to the target network
                
                # If test_policy is True, we check every target_net_update_setps if the net has learned.+
                # the net has learnet if it can obtain the maximum score 10 times in a row following its policy (with T=0)
                # or test_policy times in a row if is int
                if test_policy:
                    policy_num = policy_num + 1
                    if type(test_policy)==int:
                        ngames = test_policy
                    else:
                        ngames = 10
                        
                    test_score = 0
                    for game in range(ngames):
                        test_score = test_score + self.play_a_game(show=False)
                    test_score = test_score/ngames
                    if not type(optunaprint)==int:
                        print('Policy {:d} | Mean SCORE: {:.2f} '.format(policy_num,test_score) )
                    if test_score >= 500:
                        has_learned = True
            # Print the final score
            if optunaprint:
                progress.set_description("{:d}/{:d} :: SCORE: {:d} - T: {:.4f}".format(optunaprint,maxplayers, score,tau))
            else:
                progress.set_description("SCORE: {:d} - T: {:.4f}".format(score,tau))
            
            if has_learned == True:
                if not optunaprint:
                    print('Learning completed at epoch: {:d}'.format(episode_num))
                env.close()
                return episode_num
            
            # For speeding up the training for the optuna gridsearch, we might want to kill the learning 
            # of models that are taking longer than the best one
            if type(optuna_epoch_limit) == int:
                if episode_num >= optuna_epoch_limit:
                    env.close()
                    return episode_num

        env.close()
        return max_epoch

## CartPole-v1

In [5]:
def optuna_qlearning(ntrials = 200, nepochs = 1000):
    ''' Function for Optuna hyperparameter tuning '''
    study = optuna.create_study(sampler=optuna.samplers.TPESampler(), direction='minimize')
    
    optimizers = [torch.optim.Adam, torch.optim.SGD]
    global player
    player = 1
    global epochs
    epochs = nepochs
    def callback(study, trial):
        ''' Callback function, saves the best model. '''
        global best_model
        # If the best model result is the current one...
        if study.best_trial == trial:
            # Saves the best model to best_model
            best_model = optmodel
    
    def optuna_train(trial):
        ''' Main function for optuna. '''
        cfg = {
          'reg': trial.suggest_loguniform('reg',1e-5, 1e-3),
          'lr' : trial.suggest_loguniform('lr', 1e-4, 1e-3),
          'optimizer': trial.suggest_categorical('optimizer',list(range(len(optimizers)))),
          'gamma' : trial.suggest_uniform('gamma', .95, 1),
          #'penality' : trial.suggest_categorical('penality', [0]),
          'init_v' : trial.suggest_loguniform('init_v', 1,10)
        }
        
        global player
        global optmodel
        global epochs
        optmodel =  FullQNets('CartPole-v1')
        optmodel.to(device)
        
        learning_epochs = optmodel.training_loop(nepochs, nn.SmoothL1Loss(), optimizers[cfg['optimizer']], lr=cfg['lr'], test_policy=10,
                                                 reg_opt = cfg['reg'], gamma=cfg['gamma'], initial_value=cfg['init_v'],
                                                 bad_state_penalty = 0, optunaprint=player, maxplayers = ntrials,
                                                 optuna_epoch_limit = epochs);
        if learning_epochs < epochs:
            epochs = learning_epochs
        
        player = player + 1
        return learning_epochs
    
    study.optimize(optuna_train, n_trials=ntrials, callbacks=[callback])
    
    optuna_best_params = {
        'reg'   : study.best_params['reg'],
        'lr'    : study.best_params['lr'],
        'optimizer'   : optimizers[study.best_params['optimizer']],
        'gamma' : study.best_params['gamma'],
        #'penality' : study.best_params['penality'],
        'init_v' : study.best_params['init_v']
    }
    
    return study, optuna_best_params, best_model


In [ ]:
study, optuna_best_params, best_player = optuna_qlearning()

In [ ]:
best_player.play_a_game()


In [ ]:
optuna.visualization.plot_optimization_history(study, target_name='Episodes to learn')


## FlappyBird

In [31]:
#################
#### CLASSES ####
#################
    
#
#          |---------> [Prediction Network (DQN)]--------
#          |                |                            \
# [INPUT]--|                | Parameter update            \___Loss
#          |               \/                            /
#          |---------> [Target Network (DQN)]------------
#
class Flappy(nn.Module):
    ''' 
    Handles all the networks, environments, and others
    '''
    def __init__(self, rgb = False, verbose = False):
        super().__init__()
        
        if not rgb:
            tempenv = flappy_bird_gym.make("FlappyBird-v0")
            self.rgb = False
            self.state_space_dim = tempenv.observation_space.shape[0]
        else:
            tempenv = flappy_bird_gym.make("FlappyBird-rgb-v0")
            self.rgb = True
            self.state_space_dim = 1
            for i in range(len(tempenv.observation_space.shape) ):
                self.state_space_dim = self.state_space_dim*tempenv.observation_space.shape[i]
        
        
        self.action_space_dim = tempenv.action_space.n
        
        if verbose:
            print('Environment informations:')
            print('  observation space:', tempenv.observation_space)
            print('  action space     :', tempenv.action_space)
            print('  nets input parameters :', self.state_space_dim)
            print('  nets output parameters:', self.action_space_dim)
        
        self.policy_net = DQN(self.state_space_dim, self.action_space_dim)
        self.target_net = DQN(self.state_space_dim, self.action_space_dim)
        self.target_net.load_state_dict(self.policy_net.state_dict())
    
    def choose_action_epsilon_greedy(self, state, epsilon):
        self.policy_net.eval()
        if epsilon > 1 or epsilon < 0:
            raise Exception('The epsilon value must be between 0 and 1')
                
        # Evaluate the network output from the current state
        with torch.no_grad():
            self.policy_net.eval()
            state = torch.tensor(state, dtype=torch.float32, device=device) # Convert the state to tensor
            net_out = self.policy_net(state)

        # Get the best action (argmax of the network output)
        best_action = int(net_out.argmax())
        # Get the number of possible actions
        action_space_dim = net_out.shape[-1]

        # Select a non optimal action with probability epsilon, otherwise choose the best action
        if random.random() < epsilon:
            # List of non-optimal actions
            non_optimal_actions = [a for a in range(action_space_dim) if a != best_action]
            # Select randomly
            action = random.choice(non_optimal_actions)
        else:
            # Select best action
            action = best_action
        
        return action, net_out.cpu().numpy()
    
    def choose_action_softmax(self, state, temperature):
        self.policy_net.to(device)
        if temperature < 0:
            raise Exception('The temperature value must be greater than or equal to 0 ')
        
        # If the temperature is 0, just select the best action using the eps-greedy policy with epsilon = 0
        if temperature == 0:
            return self.choose_action_epsilon_greedy(state, 0)
    
        # Evaluate the network output from the current state
        with torch.no_grad():
            self.policy_net.eval()
            state = torch.tensor(state, dtype=torch.float32, device=device)
            net_out = self.policy_net(state)

        # Apply softmax with temp
        temperature = max(temperature, 1e-8) # set a minimum to the temperature for numerical stability
        softmax_out = nn.functional.softmax(net_out / temperature, dim=0).cpu().numpy()
                
        # Sample the action using softmax output as mass pdf
        all_possible_actions = np.arange(0, softmax_out.shape[-1])
        action = np.random.choice(all_possible_actions, p=softmax_out) # this samples a random element from "all_possible_actions" with the probability distribution p (softmax_out in this case)
    
        return action, net_out.cpu().numpy()
    
    def update_step(self, replay_mem, gamma, optimizer, loss_fn, batch_size):        
        self.policy_net.to(device)
        self.target_net.to(device)
        # Sample the data from the replay memory
        batch = replay_mem.sample(batch_size)
        batch_size = len(batch)

        # Create tensors for each element of the batch
        states      = torch.tensor([s[0] for s in batch], dtype=torch.float32, device=device)
        actions     = torch.tensor([s[1] for s in batch], dtype=torch.int64, device=device)
        rewards     = torch.tensor([s[3] for s in batch], dtype=torch.float32, device=device)

        # Compute a mask of non-final states (all the elements where the next state is not None)
        non_final_next_states = torch.tensor([s[2] for s in batch if s[2] is not None], dtype=torch.float32, device=device) # the next state can be None if the game has ended
        non_final_mask = torch.tensor([s[2] is not None for s in batch], dtype=torch.bool, device=device)

        # Compute all the Q values (forward pass)
        self.policy_net.train()
        q_values = self.policy_net(states)
        # Select the proper Q value for the corresponding action taken Q(s_t, a)
        state_action_values = q_values.gather(1, actions.unsqueeze(1))
        
        # Compute the value function of the next states using the target network V(s_{t+1}) = max_a( Q_target(s_{t+1}, a)) )
        with torch.no_grad():
            self.target_net.eval()
            q_values_target = self.target_net(non_final_next_states)
        next_state_max_q_values = torch.zeros(batch_size, device=device)
        next_state_max_q_values[non_final_mask] = q_values_target.max(dim=1)[0]

        # Compute the expected Q values
        expected_state_action_values = rewards + (next_state_max_q_values * gamma)
        expected_state_action_values = expected_state_action_values.unsqueeze(1) # Set the required tensor shape

        # Compute the Huber loss
        loss = loss_fn(state_action_values, expected_state_action_values)

        # Optimize the model
        optimizer.zero_grad()
        loss.backward()
        # Apply gradient clipping (clip all the gradients greater than 2 for training stability)
        nn.utils.clip_grad_norm_(self.policy_net.parameters(), 2)
        optimizer.step()
        
        return optimizer
    
    def play_a_game(self, show = False, debug = False):
        # Initialize the Gym environmentif not rgb:
        if self.rgb:
            env = flappy_bird_gym.make("FlappyBird-rgb-v0")
        else:
            env = flappy_bird_gym.make("FlappyBird-v0")
        
        # Reset the environment and get the initial state
        state = env.reset()
        # Reset the score. The final score will be the total amount of steps before the pole falls
        score = 0
        done = False
        # Go on until the pole falls off or the score reach 490
        while not done:
            # Choose the best action (temperature 0)
            action, q_values = self.choose_action_softmax(state, temperature=0)
            if debug:
                print('---------------')
                print('action', action)
                print('state', state)
            # Apply the action and get the next state, the reward and a flag "done" that is True if the game is ended
            next_state, reward, done, info = env.step(action)
            # Visually render the environment
            if show:
                env.render()
                time.sleep(1 / 20)  # FPS
            # Update the final score (+1 for each step)
            score += reward 
            # Set the current state for the next iteration
            state = next_state
            # Check if the episode ended (the pole fell down)
        # Print the final score
        if show:
            print(f"SCORE: {score}") 
        env.close()
        
        return score, state
    
    def training_loop(self, max_epoch, loss_fn, optimizer_fn, initial_value=5, gamma = 0.97, 
                 replay_memory_capacity = 10000, lr = 1e-2, reg_opt = 1e-5, 
                 target_net_update_steps = 10, bad_state_penalty = 0, test_policy = False,
                 batch_size = 128, min_samples_for_training = 1000, render=False, optunaprint = False, maxplayers = 0,
                 optuna_epoch_limit = False):
        '''
        PARAMETERS
        gamma: gamma parameter for the long term reward
        replay_memory_capacity: Replay memory capacity
        lr: Optimizer learning rate
        loss_fn: Loss function
        optimizer_fn: Optimizer function
        target_net_update_steps: Number of episodes to wait before updating the target network
        batch_size: Number of samples to take from the replay memory for each update
        bad_state_penalty: Penalty to the reward when we are in a bad state (in this case when the pole falls down) 
        min_samples_for_training: Minimum samples in the replay memory to enable the training
        test_policy: if True, for every target_net_update_steps steps, it will play 10 games following its policy (T=0)
                     and if it obtains the maximum score (500) it will exit
                     if is int > 0 it will do the same playing test_policy number of games
        '''
        
        if self.rgb:
            env = flappy_bird_gym.make("FlappyBird-rgb-v0")
        else:
            env = flappy_bird_gym.make("FlappyBird-v0")
            
        optimizer = optimizer_fn(self.policy_net.parameters(), lr=lr, weight_decay=reg_opt)
        has_learned = False # Boolean variable, if test_policy == True and has obtained the max score on the test games
                            # it will exit learning
        # We compute the exponential decay in such a way the shape of the exploration 
        # profile does not depend on the number of iterations
        exp_decay = np.exp(-np.log(initial_value) / max_epoch * 6) 
        exploration_profile = [initial_value * (exp_decay ** i) for i in range(max_epoch)]
        
        ### Initialize the replay memory
        replay_mem = ReplayMemory(replay_memory_capacity)   
        
        env.seed(0)
        policy_num = 0
        progress = tqdm(exploration_profile)
        #print('n iterations:', len(exploration_profile) )
        for episode_num, tau in enumerate(progress):
            # Reset the environment and get the initial state
            state = env.reset()
            
            # Reset the score. The final score will be the total amount of steps before the pole falls
            score = 0
            
            done = False
            
            # Go on until the pole falls off
            while not done:
                # Choose the action following the policy
                action, q_values = self.choose_action_softmax(state, temperature=tau)
                
                # Apply the action and get the next state, the reward and a flag "done" 
                # that is True if the game is ended
                next_state, reward, done, info = env.step(action)
                
                # We apply a (linear) penalty when the cart is far from center
                pos_weight = 5
                reward = reward - pos_weight * np.abs(state[1])
                
                # Update the final score (+1 for each step)
                score += 1
                
                # Update the replay memory
                replay_mem.push(state, action, next_state, reward)

                # Update the network
                if len(replay_mem) > min_samples_for_training: # we enable the training only if we have enough samples in the replay memory, otherwise the training will use the same samples too often
                    optimizer = self.update_step(replay_mem, gamma, optimizer, loss_fn, batch_size)
                
                if render:
                    # Visually render the environment (disable to speed up the training)
                    env.render()
                    time.sleep(1 / 30)  # FPS

                # Set the current state for the next iteration
                state = next_state

            # Update the target network every target_net_update_steps episodes
            if episode_num % target_net_update_steps == 0:
                self.target_net.load_state_dict(self.policy_net.state_dict()) # This will copy the weights of the policy network to the target network
                
                # If test_policy is True, we check every target_net_update_setps if the net has learned.+
                # the net has learnet if it can obtain the maximum score 10 times in a row following its policy (with T=0)
                # or test_policy times in a row if is int
                if test_policy:
                    policy_num = policy_num + 1
                    if type(test_policy)==int:
                        ngames = test_policy
                    else:
                        ngames = 10
                        
                    test_score = 0
                    for game in range(ngames):
                        test_score = test_score + self.play_a_game(show=False)
                    test_score = test_score/ngames
                    if not type(optunaprint)==int:
                        print('Policy {:d} | Mean SCORE: {:.2f} '.format(policy_num,test_score) )
                    if test_score >= 500:
                        has_learned = True
            # Print the final score
            if optunaprint:
                progress.set_description("{:d}/{:d} :: SCORE: {:d} - T: {:.4f}".format(optunaprint,maxplayers, score,tau))
            else:
                progress.set_description("SCORE: {:d} - T: {:.4f}".format(score,tau))
            
            if has_learned == True:
                if not optunaprint:
                    print('Learning completed at epoch: {:d}'.format(episode_num))
                env.close()
                return episode_num
            
            # For speeding up the training for the optuna gridsearch, we might want to kill the learning 
            # of models that are taking longer than the best one
            if type(optuna_epoch_limit) == int:
                if episode_num >= optuna_epoch_limit:
                    env.close()
                    return episode_num

        env.close()
        return max_epoch

### FlappyBird-v0

![img1](https://raw.githubusercontent.com/Talendar/flappy-bird-gym/main/imgs/observations.png)

* ***observation space***: yields simple numerical information about the game's state as observations. The yielded attributes are the:
  * horizontal distance to the next pipe;
  * difference between the player's y position and the next hole's y position
* ***actionspace***:
  * tap: flutter
  * do nothing

In [34]:
player =  Flappy(verbose = True)

Environment informations:
  observation space: Box(-inf, inf, (2,), float32)
  action space     : Discrete(2)
  nets input parameters : 2
  nets output parameters: 2


In [35]:
player.training_loop(2000, nn.SmoothL1Loss(), torch.optim.SGD, lr=1e-2, initial_value = 5)

  0%|          | 0/2000 [00:00<?, ?it/s]

/tmp/ipykernel_5232/2515109324.py:105: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1639180487213/work/torch/csrc/utils/tensor_new.cpp:201.)
  states      = torch.tensor([s[0] for s in batch], dtype=torch.float32, device=device)


KeyboardInterrupt: 

In [ ]:
# To test how good it can play, it will play 10 games and average the score:
games = 10

mean_score = 0
for game in range(games):
    score = player.play_a_game(show = False)
    print('GAME {:d}: SCORE {:d}'.format(game+1,score) )
    
    mean_score = mean_score + score

mean_score = mean_scores/games
print('____________________')
print('MEAN SCORE: {:2.f}'.format(mean_score))


In [27]:
# Lets see once how it plays
player.play_a_game(show = True)


SCORE: 1690


1690

### FlappyBird-rgb-v0

![img2](https://upload.wikimedia.org/wikipedia/it/thumb/c/c7/Flappy_Bird_gameplay.jpeg/135px-Flappy_Bird_gameplay.jpeg)

* ***observation space***: Whole RGB image, this means a 3 channel image of resolution 228x512 (228x512x3)
* ***actionspace***:
  * tap: flutter
  * do nothing

In [32]:
player =  Flappy(rgb = True, verbose = True)

Environment informations:
  observation space: Box(0.0, 255.0, (288, 512, 3), float32)
  action space     : Discrete(2)
  nets input parameters : 442368
  nets output parameters: 2


In [33]:
score, state = player.play_a_game(show=True)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (147456x3 and 442368x128)

In [10]:
player.observation_space.shape[0]

AttributeError: 'Flappy' object has no attribute 'observation_space'

In [18]:
tempenv = flappy_bird_gym.make("FlappyBird-rgb-v0")

In [22]:
tempenv.observation_space.shape

(288, 512, 3)

In [23]:
288*512*3

442368

In [29]:
tempenv.observation_space.shape[0]

288

In [30]:

dim = 1
for i in range(len(tempenv.observation_space.shape) ):
    dim = dim*tempenv.observation_space.shape[i]
    
print(dim)

442368


---
## Resources:
* [Playing Atari with Deep Reinforcement Learning](https://www.cs.toronto.edu/~vmnih/docs/dqn.pdf)